In [ ]:
# Manage imports
from pathlib import Path
import folium
import AisAudioLabeler as aal
import LabelerUtilities as lu

In [ ]:
# Assign data home directory, collection filename
data_home = Path("~").expanduser() / "Data" / "AISonobuoy"
collection_filename = "collection-ais.json"

In [ ]:
# Load file describing the collection
collection_path = Path(data_home) / collection_filename
collection = lu.load_json_file(collection_path)

In [ ]:
# For now, assume a single source and hydrophone
if len(collection["sources"]) > 1 or len(collection["hydrophones"]) > 1:
    raise Exception("Only one source and one hydrophone expected")
source = collection["sources"][0]
hydrophone = collection["hydrophones"][0]

In [ ]:
# Load AIS data
ais = aal.get_ais_dataframe(data_home, source)

In [ ]:
# Plot the location of the hydrophone and ship with the specified mmsi
# See:
#    https://python-visualization.github.io/folium/quickstart.html
#    https://snyk.io/advisor/npm-package/leaflet
m = folium.Map(location=[hydrophone["lat"], hydrophone["lon"]], zoom_start=13)
folium.Circle(
    radius=0,
    location=[hydrophone["lat"], hydrophone["lon"]],
    popup="hydrophone",
    color="black",
    fill=True,
    ).add_to(m)
mmsi = "367293000"
ais_s = ais.loc[ais["mmsi"] == mmsi]
for index, row_s in ais_s.iterrows():
    if row_s["status"] == "UnderWayUsingEngine":
        color = "orange"
    else:
        color = "cadetblue"
    folium.Circle(
        radius=0,
        location=[row_s["lat"], row_s["lon"]],
        popup=f"{row_s['mmsi']}\n{row_s['shiptype']}",
        color=color,
        fill=False,
    ).add_to(m)

In [ ]:
ais_s

In [ ]:
# Display the map
m

In [ ]:
# Save the map
m.save(data_home / f"map-{source['name']}-{source['prefix']}-{mmsi}.html")